## 1. 수집 테스트

### 벅스뮤직 및 빌보드재팬 차트 곡목록 수집 자동화

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
!pip install selenium tqdm pandas
!apt-get update
!apt-get install -y chromium-browser xvfb

import time
import random
import pandas as pd
import os
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==============================================================================
# 2. Colab 환경을 위한 셀레니움 설정
# ==============================================================================
def setup_driver():
    """Colab에서 Selenium을 사용하기 위한 WebDriver를 설정하고 반환합니다."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    return driver

# ==============================================================================
# 3. 수집 대상 날짜 및 URL 생성
# ==============================================================================
def get_target_urls(start_year, end_year):
    """지정된 기간 동안 각 분기 첫째 주에 해당하는 차트 URL 목록을 생성합니다."""
    urls = []
    for year in range(start_year, end_year + 1):
        for month in [1, 4, 7, 10]:
            if (year == start_year and month < 7) or (year == end_year and month > 7):
                continue

            first_day = datetime(year, month, 1)
            days_to_add = (7 - first_day.weekday()) % 7
            first_monday = first_day + timedelta(days=days_to_add)

            # 빌보드 재팬: 날짜 기준을 월요일로 설정
            billboard_date = first_monday
            urls.append({
                'site': 'BillboardJp',
                'date_key': billboard_date.strftime('%Y-%m-%d'),
                'url': f"https://www.billboard-japan.com/charts/detail?a=hot100&year={billboard_date.year}&month={billboard_date.month:02d}&day={billboard_date.day:02d}"
            })

            # 벅스뮤직: 날짜 기준을 일요일로 설정
            bugs_date = first_monday + timedelta(days=6)
            urls.append({
                'site': 'Bugs',
                'date_key': bugs_date.strftime('%Y-%m-%d'),
                'url': f"https://music.bugs.co.kr/chart/track/week/njpop?chartdate={bugs_date.strftime('%Y%m%d')}"
            })
    return urls

# ==============================================================================
# 4. 각 사이트별 데이터 추출(파싱) 함수 (이전과 동일)
# ==============================================================================
def parse_billboard(driver, date_key):
    results = []
    rows = driver.find_elements(By.XPATH, "//table//tr[contains(@class, 'rank')]")
    for row in rows:
        try:
            rank = row.find_element(By.CSS_SELECTOR, "td.rank_td span").text.strip()
            title = row.find_element(By.CSS_SELECTOR, "p.musuc_title").text.strip()
            artist = row.find_element(By.CSS_SELECTOR, "p.artist_name").text.strip()
            results.append(['BillboardJp', date_key, rank, title, artist])
        except NoSuchElementException: continue
    return results

def parse_bugs(driver, date_key):
    results = []
    rows = driver.find_elements(By.XPATH, "//table[contains(@class, 'trackList')]//tbody/tr")
    for row in rows:
        try:
            rank = row.find_element(By.CSS_SELECTOR, "div.ranking strong").text.strip()
            title = row.find_element(By.CSS_SELECTOR, "p.title a").text.strip()
            artist = row.find_element(By.CSS_SELECTOR, "p.artist a").text.strip()
            results.append(['Bugs', date_key, rank, title, artist])
        except NoSuchElementException: continue
    return results

# ==============================================================================
# 5. 메인 크롤링 실행 로직
# ==============================================================================
output_filename = 'jpop_chart_list.csv'
all_urls = get_target_urls(2020, 2025)
print(f"✅ 총 {len(all_urls)}개의 차트를 수집할 예정입니다. (빌보드, 벅스)")

if os.path.exists(output_filename):
    print(f"📄 기존 파일 '{output_filename}'을 발견했습니다. 이어서 수집을 시작합니다.")
    existing_df = pd.read_csv(output_filename)
    completed_keys = (existing_df['site'].astype(str) + existing_df['date_key'].astype(str)).tolist()
    urls_to_scrape = [u for u in all_urls if u['site'] + u['date_key'] not in completed_keys]
    print(f"⏭️ {len(all_urls) - len(urls_to_scrape)}개 차트는 이미 수집되어 건너뜁니다.")
else:
    print(f"✨ 새 파일 '{output_filename}'을 생성하고 수집을 시작합니다.")
    urls_to_scrape = all_urls
    pd.DataFrame(columns=['site', 'date_key', 'rank', 'title', 'artist']).to_csv(output_filename, index=False, encoding='utf-8-sig')

if not urls_to_scrape:
    print("🎉 모든 차트 수집이 이미 완료되었습니다!")
else:
    # ✨ [변경점] for 루프 밖에서는 driver를 생성하지 않음
    for task in tqdm(urls_to_scrape, desc="J-POP 차트 수집 중"):
        driver = None # 루프 시작 시 driver 초기화
        site, url, date_key = task['site'], task['url'], task['date_key']
        try:
            # ✨ [변경점] 각 URL마다 새로운 드라이버 생성
            driver = setup_driver()
            wait = WebDriverWait(driver, 15) # 대기 시간을 15초로 약간 늘림

            driver.get(url)

            if site == 'BillboardJp':
                wait.until(EC.presence_of_element_located((By.XPATH, "//table//tr[contains(@class, 'rank')]")))
                scraped_data = parse_billboard(driver, date_key)
            elif site == 'Bugs':
                wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'trackList')]//tbody/tr")))
                scraped_data = parse_bugs(driver, date_key)

            if not scraped_data:
                print(f"\n⚠️ 경고: {site} ({date_key}) 에서 데이터를 찾지 못했습니다.")
                continue

            first_song = scraped_data[0]
            summary = (f"✅ {first_song[0]} ({first_song[1]}) 차트 수집 완료. ({len(scraped_data)}곡) | 1위: {first_song[3]} - {first_song[4]}")
            print(f"\n{summary}")
            temp_df = pd.DataFrame(scraped_data, columns=['site', 'date_key', 'rank', 'title', 'artist'])
            temp_df.to_csv(output_filename, mode='a', header=False, index=False, encoding='utf-8-sig')

        except Exception as e:
            print(f"\n❌ 루프 내 오류 발생: {site} ({date_key}) 처리 중 문제 발생. 오류: {e}")

        finally:
            # ✨ [변경점] 한 작업이 끝나면 반드시 드라이버 종료
            if driver:
                driver.quit()

    print("\n🎉 수집 작업이 모두 종료되었습니다.")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

J-POP 차트 수집 중:   0%|          | 0/24 [00:00<?, ?it/s]


✅ BillboardJp (2022-10-03) 차트 수집 완료. (100곡) | 1위: 新時代 (ウタ from ONE PIECE FILM RED) - Ado

✅ Bugs (2022-10-09) 차트 수집 완료. (94곡) | 1위: Shinunoga E-Wa - Fujii Kaze

✅ BillboardJp (2023-01-02) 차트 수집 완료. (100곡) | 1위: Subtitle - Official髭男dism

✅ Bugs (2023-01-08) 차트 수집 완료. (95곡) | 1위: KICK BACK - Kenshi Yonezu(켄시 요네즈/米津 玄師)

✅ BillboardJp (2023-04-03) 차트 수집 완료. (100곡) | 1위: Bye-Bye Show - BiSH

✅ Bugs (2023-04-09) 차트 수집 완료. (98곡) | 1위: NIGHT DANCER - imase

✅ BillboardJp (2023-07-03) 차트 수집 완료. (100곡) | 1위: アイドル - YOASOBI

✅ Bugs (2023-07-09) 차트 수집 완료. (98곡) | 1위: アイドル (애니메이션 '최애의 아이' 1기 오프닝곡) - YOASOBI

✅ BillboardJp (2023-10-02) 차트 수집 완료. (100곡) | 1위: 唱 - Ado

✅ Bugs (2023-10-08) 차트 수집 완료. (97곡) | 1위: アイドル (애니메이션 '최애의 아이' 1기 오프닝곡) - YOASOBI

✅ BillboardJp (2024-01-01) 차트 수집 완료. (100곡) | 1위: 唱 - Ado

✅ Bugs (2024-01-07) 차트 수집 완료. (100곡) | 1위: Lemon - Kenshi Yonezu(켄시 요네즈/米津 玄師)

✅ BillboardJp (2024-04-01) 차트 수집 완료. (100곡) | 1위: Bling-Bang-Bang-Born - Creepy Nuts

✅ Bugs (2024-04-07) 차트 수집 완

### 곡 제목 중복패턴분석

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
!pip install thefuzz python-Levenshtein

import pandas as pd
import re
from thefuzz import fuzz
from itertools import combinations
from collections import Counter
from tqdm.auto import tqdm

# ==============================================================================
# 2. 데이터 처리 함수 정의
# ==============================================================================
def normalize_text(text):
    """문자열 비교를 위해 텍스트를 정규화하는 함수"""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[\(（\[【].*?[\)）\]】]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = " ".join(text.split())
    return text

# ==============================================================================
# 3. 메인 분석 로직
# ==============================================================================
# 원본에 가까운 unique 리스트로 분석을 다시 시작합니다.
input_file = 'jpop_unique_song_list.csv'

try:
    df = pd.read_csv(input_file)
    print(f"📄 '{input_file}' 파일 로드 완료. 분석 대상 곡 수: {len(df)}개")
    print("-" * 50)

    # 비교를 위한 정규화된 아티스트 및 제목 열 생성
    df['norm_artist'] = df['artist'].apply(normalize_text)
    df['norm_title'] = df['title'].apply(normalize_text)

    # --- 1. 애매한 중복 후보 목록 추출 (유사도 75-89점) ---
    ambiguous_pairs = []
    grouped = df.groupby('norm_artist')

    print("🔄 동일 아티스트 내 애매한 중복 후보를 탐색합니다 (유사도 75-89점)...")
    for artist, group in tqdm(grouped, desc="아티스트별 비교 중"):
        if len(group) < 2:
            continue

        for idx1, idx2 in combinations(group.index, 2):
            title1 = group.loc[idx1, 'norm_title']
            original_title1 = group.loc[idx1, 'title']

            title2 = group.loc[idx2, 'norm_title']
            original_title2 = group.loc[idx2, 'title']

            if not title1 or not title2:
                continue

            similarity = fuzz.ratio(title1, title2)

            if 75 <= similarity < 90:
                ambiguous_pairs.append({
                    'artist': group.loc[idx1, 'artist'],
                    'title_1': original_title1,
                    'title_2': original_title2,
                    'similarity': similarity
                })

    print("\n--- [분석 1] 애매한 중복 후보 목록 ---")
    if ambiguous_pairs:
        ambiguous_df = pd.DataFrame(ambiguous_pairs)
        print(ambiguous_df.to_string())
    else:
        print("애매한 중복 후보(유사도 75-89점)가 발견되지 않았습니다.")
    print("-" * 50)

    # --- 2. 가장 많이 등장한 아티스트 TOP 20 확인 ---
    print("\n--- [분석 2] 가장 많이 등장한 아티스트 TOP 20 (원본 이름) ---")
    print(df['artist'].value_counts().head(20))
    print("\n--- [분석 3] 가장 많이 등장한 아티스트 TOP 20 (정규화된 이름) ---")
    print(df['norm_artist'].value_counts().head(20))
    print("-" * 50)

    print("\n✅ 분석 완료. 위 목록들을 검토하여 추가로 통합해야 할 아티스트나 제목의 패턴을 찾아보십시오.")
    print("예: 'OFFICIAL HIGE DANDISM'과 'official hige dandism'이 따로 집계된다면, 아티스트 이름 표기법 통일이 필요합니다.")


except FileNotFoundError:
    print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

📄 'jpop_unique_song_list.csv' 파일 로드 완료. 분석 대상 곡 수: 1475개
--------------------------------------------------
🔄 동일 아티스트 내 애매한 중복 후보를 탐색합니다 (유사도 75-89점)...


아티스트별 비교 중:   0%|          | 0/532 [00:00<?, ?it/s]


--- [분석 1] 애매한 중복 후보 목록 ---
                        artist                                                           title_1                                           title_2  similarity
0                          ALI  LOST IN PARADISE (Jujutsu Kaisen Ending Theme Song) (feat. AKLO)                       LOST IN PARADISE feat. AKLO          76
1                       DISH//                                           猫 〜THE FIRST TAKE ver.〜                      Neko -The First Take Version          81
2  Kenshi Yonezu(켄시 요네즈/米津 玄師)                                      Haiirotoao ( + Masaki Suda )     灰色と青 (with Masaki Suda) / Haiirotoao (잿빛과 푸름)          80
3       Oku Hanako(오쿠 하나코/奧華子)                                                變わらないもの (변하지 않는 것)               変わらないもの (Kawaranai Mono / 변하지 않는 것)          86
4              RADWIMPS(래드윔프스)                  なんでもないや (movie ver.) / Nandemonaiya (movie ver.)                         Nandemonaiya (movie ver.)          75
5              RA

### 곡목록 중복제거

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
!pip install pandas thefuzz python-Levenshtein networkx pykakasi -q

import pandas as pd
import re
from thefuzz import fuzz
from itertools import combinations
from tqdm.auto import tqdm
import networkx as nx
import pykakasi

# ==============================================================================
# 2. 데이터 처리 함수 정의
# ==============================================================================
kks = pykakasi.kakasi()

def normalize_artist_name(name):
    """아티스트 이름을 비교 가능한 여러 형태로 변환하는 함수"""
    if not isinstance(name, str):
        return {'basic': '', 'romaji': ''}

    basic_norm = name.lower()
    basic_norm = re.sub(r'[\(（\[【].*?[\)）\]】]', '', basic_norm).strip()

    # pykakasi는 한자/가나를 로마자로 변환합니다.
    romaji_result = kks.convert(basic_norm)
    romaji_norm_str = "".join([item['hepburn'] for item in romaji_result])
    romaji_norm_str = re.sub(r'[^a-z0-9]', '', romaji_norm_str)

    return {'basic': basic_norm, 'romaji': romaji_norm_str}

def normalize_title(title):
    """곡 제목 정규화 함수"""
    if not isinstance(title, str):
        return ""
    text = title.lower()
    text = re.sub(r'[\(（\[【].*?[\)）\]】]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = " ".join(text.split())
    return text

# ==============================================================================
# 3. 메인 중복 처리 로직
# ==============================================================================
# ✨ [핵심 변경점] 모든 데이터가 포함된 원본 크롤링 파일로 변경
input_file = 'jpop_chart_list (1).csv'
output_file = 'jpop_final_unique_list.csv'

try:
    df = pd.read_csv(input_file)
    print(f"📄 원본 파일 '{input_file}' 로드 완료. 전체 데이터 수: {len(df)}개")

    # --- 사전 분석: 원본 데이터의 아티스트 이름 현황 ---
    print("\n--- [사전 분석] 원본 데이터의 아티스트 상위 20명 (중복 제거 전) ---")
    print(df['artist'].value_counts().head(20))
    print("-" * 50)

    # --- 1. 아티스트 이름 군집화 ---
    artist_counts = df['artist'].value_counts().to_dict()
    unique_original_artists = list(artist_counts.keys())

    print(f"🔄 {len(unique_original_artists)}개의 고유 아티스트 표기를 대상으로 군집화를 시작합니다...")

    artist_norms = {name: normalize_artist_name(name) for name in unique_original_artists}

    G = nx.Graph()
    G.add_nodes_from(unique_original_artists)
    SIMILARITY_THRESHOLD = 85

    for artist1, artist2 in tqdm(combinations(unique_original_artists, 2), desc="유사도 관계망 생성 중", total=len(unique_original_artists)*(len(unique_original_artists)-1)//2):
        norm1 = artist_norms[artist1]
        norm2 = artist_norms[artist2]

        if (fuzz.ratio(norm1['basic'], norm2['basic']) >= SIMILARITY_THRESHOLD or
            (norm1['romaji'] and norm2['romaji'] and fuzz.ratio(norm1['romaji'], norm2['romaji']) >= SIMILARITY_THRESHOLD)):
            G.add_edge(artist1, artist2)

    clusters = list(nx.connected_components(G))
    print(f"✨ {len(clusters)}개의 아티스트 군집을 생성했습니다.")

    # --- 2. 대표 이름 선정 및 매핑 생성 ---
    artist_to_canonical_map = {}
    for cluster in clusters:
        canonical_name = max(cluster, key=lambda name: artist_counts.get(name, 0))
        for name in cluster:
            artist_to_canonical_map[name] = canonical_name

    # --- 3. '대표 아티스트'와 '정제된 제목' 열 추가 ---
    df['canonical_artist'] = df['artist'].map(artist_to_canonical_map).fillna(df['artist'])
    df['normalized_title'] = df['title'].apply(normalize_title)

    # --- 4. 최종 중복 제거 ---
    df_final = df.sort_values(
        by=['site', 'date_key', 'rank'],
        ascending=[True, False, True]
    ).drop_duplicates(subset=['canonical_artist', 'normalized_title'], keep='first')

    df_final = df_final.drop(columns=['canonical_artist', 'normalized_title'])

    print(f"\n📊 최종 고유 곡 수: {len(df_final)}개 (제거된 곡: {len(df) - len(df_final)}개)")

    # --- 5. 최종 결과 저장 ---
    df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"✅ '{output_file}' 파일에 최종 결과를 저장했습니다.")

    print("\n--- 최종 고유 곡 목록 샘플 (상위 10개) ---")
    print(df_final.head(10))

except FileNotFoundError:
    print(f"오류: '{input_file}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.6 MB/s eta 0:00:00
📄 원본 파일 'jpop_chart_list (1).csv' 로드 완료. 전체 데이터 수: 4107개

--- [사전 분석] 원본 데이터의 아티스트 상위 20명 (중복 제거 전) ---
artist
YOASOBI                        208
Kenshi Yonezu(켄시 요네즈/米津 玄師)    201
OFFICIAL HIGE DANDISM          168
Mrs.GREEN APPLE                156
RADWIMPS(래드윔프스)                143
aimyon                         139
Official髭男dism                 106
Vaundy                          94
Ado                             86
back number                     80
Yuuri                           78
Yorushika                       72
優里                              72
米津玄師                            60
あいみょん                           59
King Gnu                        54
King Gnu(킹 누)                   51
SPYAIR(스파이에어/スパイエアー

유사도 관계망 생성 중:   0%|          | 0/150975 [00:00<?, ?it/s]

✨ 509개의 아티스트 군집을 생성했습니다.

📊 최종 고유 곡 수: 1439개 (제거된 곡: 2668개)
✅ 'jpop_final_unique_list.csv' 파일에 최종 결과를 저장했습니다.

--- 최종 고유 곡 목록 샘플 (상위 10개) ---
             site    date_key  rank           title           artist
3907  BillboardJp  2025-07-07     1   Make or Break             櫻坂46
3908  BillboardJp  2025-07-07     2       DIFFERENT      LE SSERAFIM
3909  BillboardJp  2025-07-07     3       breakfast  Mrs.GREEN APPLE
3910  BillboardJp  2025-07-07     4            クスシキ  Mrs.GREEN APPLE
3911  BillboardJp  2025-07-07     5          Plazma             米津玄師
3912  BillboardJp  2025-07-07     6            ROSE             HANA
3913  BillboardJp  2025-07-07     7  Burning Flower             HANA
3914  BillboardJp  2025-07-07     8  Cheek to Cheek             IMP.
3915  BillboardJp  2025-07-07     9              夢中         BE:FIRST
3916  BillboardJp  2025-07-07    10              怪獣          サカナクション


### 벅스뮤직 단일곡 크롤링 테스트

In [ ]:
# Colab 환경에서 셀레니움을 실행하기 위한 필수 라이브러리 설치
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser xvfb

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# ==============================================================================
# 1. Colab 환경을 위한 셀레니움 설정
# ==============================================================================
# Chrome 옵션을 설정하여 GUI 없이 백그라운드에서 실행 (헤드리스 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("window-size=1920x1080")

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

# 크롤링할 URL
url = "https://music.bugs.co.kr/track/4905308"

try:
    # ==============================================================================
    # 2. 웹사이트 접속 및 정보 크롤링
    # ==============================================================================
    # 지정한 URL로 접속
    driver.get(url)

    # 웹페이지가 완전히 로드될 때까지 잠시 대기 (필요에 따라 시간 조정)
    time.sleep(2)

    # --- 곡 제목 추출 ---
    # XPath를 사용하여 제목이 들어있는 <h1> 요소를 찾음
    title_element = driver.find_element(By.XPATH, '//*[@id="container"]/header/div/h1')
    title = title_element.text.strip()

    # --- 아티스트 정보 추출 ---
    # 아티스트 정보가 있는 테이블 영역에서 모든 아티스트 <a> 요소를 찾음
    artist_elements = driver.find_elements(By.XPATH, '//table[@class="info"]//th[text()="아티스트"]/following-sibling::td//a')
    # 리스트 컴프리헨션을 사용하여 각 요소의 텍스트(아티스트명)를 추출
    artists = [artist.text.strip() for artist in artist_elements]
    # 쉼표로 구분된 하나의 문자열로 합침
    artist_string = ", ".join(artists)

    # --- 가사 추출 ---
    # 가사가 들어있는 <xmp> 요소를 찾음
    lyrics_element = driver.find_element(By.XPATH, '//div[@class="lyricsContainer"]/xmp')
    # get_attribute("innerHTML")을 사용하여 줄바꿈이 포함된 원본 텍스트를 가져옴
    lyrics = lyrics_element.get_attribute("innerHTML").strip()


    # ==============================================================================
    # 3. 수집된 결과 출력
    # ==============================================================================
    print("✅ 벅스 뮤직 크롤링 결과\n")
    print("="*30)
    print(f"🎵 제목: {title}")
    print("="*30)
    print(f"🎤 아티스트: {artist_string}")
    print("="*30)
    print("📜 가사:")
    print(lyrics)
    print("="*30)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

finally:
    # 모든 작업이 끝나면 브라우저 세션을 종료하여 리소스를 정리
    driver.quit()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

### Uta-net 단일 곡 수집 테스트

In [ ]:
# Colab 환경에서 selenium과 웹드라이버 관련 라이브러리를 설치합니다.
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Chrome 옵션을 설정합니다. Colab 환경에서는 '--headless', '--no-sandbox', '--disable-dev-shm-usage' 옵션이 필요합니다.
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 웹드라이버를 초기화합니다.
driver = webdriver.Chrome(options=options)

try:
    # 1. 검색어 "愛を伝えたいだとか"의 검색 결과 페이지로 이동합니다.
    search_url = "https://www.uta-net.com/search/?Keyword=%E6%84%9B%E3%82%92%E4%BC%9D%E3%81%88%E3%81%9F%E3%81%84%E3%81%A0%E3%81%A8%E3%81%8B&Aselect=2&Bselect=3"
    driver.get(search_url)
    print(f"접속 완료: {search_url}")

    # 페이지가 로드될 때까지 잠시 대기합니다.
    time.sleep(3)

    # 2. 검색 결과 페이지에서 첫 번째 곡의 가사 페이지 링크를 XPath를 이용해 찾습니다.
    #    제공된 XPath는 span 태그를 가리키지만, 실제 링크는 부모인 a 태그에 있으므로 수정합니다.
    song_link_xpath = "/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/table/tbody/tr/td[1]/a"
    song_link_element = driver.find_element(By.XPATH, song_link_xpath)

    # 링크 URL을 가져와서 이동합니다.
    lyrics_page_url = song_link_element.get_attribute('href')
    print(f"가사 페이지로 이동: {lyrics_page_url}")
    driver.get(lyrics_page_url)

    # 가사 페이지가 로드될 때까지 잠시 대기합니다.
    time.sleep(3)

    # 3. 가사 텍스트가 있는 요소의 XPath를 사용하여 가사를 추출합니다.
    lyrics_xpath = "//*[@id='kashi_area']"
    lyrics_element = driver.find_element(By.XPATH, lyrics_xpath)
    lyrics_text = lyrics_element.text

    # 4. 추출된 가사를 출력합니다.
    print("\n--- 가사 추출 결과 ---")
    print(lyrics_text)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

finally:
    # 드라이버를 종료하여 리소스를 정리합니다.
    driver.quit()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

### 곡목록 내 순회하며 각곡 가사 수집 자동화 파이프라인(벅스뮤직 및 Uta-net)

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
!pip install selenium tqdm pandas python-Levenshtein thefuzz
!apt-get update
!apt-get install -y chromium-browser xvfb

import time
import random
import pandas as pd
import os
import re
from urllib.parse import urljoin, quote
from tqdm.auto import tqdm

# --- Selenium 관련 임포트 ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# --- 문자열 유사도 검증 라이브러리 임포트 ---
from thefuzz import fuzz

# ==============================================================================
# 2. Colab 환경을 위한 셀레니움 및 기본 설정
# ==============================================================================
# --- 파일 및 설정 정의 ---
INPUT_FILE = 'jpop_chart_list.csv'
UNIQUE_SONG_LIST_FILE = 'jpop_unique_song_list.csv'
OUTPUT_FILE = 'jpop_lyrics_collection.csv'
FAILED_FILE = 'failed_list.txt'
SIMILARITY_THRESHOLD = 65 # 유사도 임계값

def setup_driver():
    """Colab에서 Selenium을 사용하기 위한 WebDriver를 설정하고 반환합니다."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # [핵심 수정] 세션 충돌 방지를 위한 옵션 추가
    # 1. 고유한 임시 사용자 디렉터리 지정
    chrome_options.add_argument(f'--user-data-dir=/tmp/chrome_user_data_{random.randint(10000, 99999)}')
    # 2. 가상 환경에서의 안정성을 위한 GPU 비활성화
    chrome_options.add_argument("--disable-gpu")

    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    return driver

def normalize_string(text):
    """문자열에서 특수문자, 공백을 제거하고 소문자화하여 비교 정확도를 높입니다."""
    text = re.sub(r'\(.*?\)|\[.*?\]', '', text)
    # 영문자, 숫자, 한글, 그리고 일본어(히라가나, 가타카나, 한자)를 모두 포함
    text = re.sub(r'[^a-zA-Z0-9\s\u3040-\u30ff\u3131-\uD79D\u4e00-\u9fff]', '', text)
    return text.lower().strip()

# ==============================================================================
# 3. 사이트별 가사 수집 함수 (수정 완료)
# ==============================================================================

def scrape_from_bugs_final_perfected(driver, artist, title):
    """[최종 수정] '가사' 탭 검색 -> '곡정보' 링크 추출 -> 상세 페이지 이동 -> 가사 수집"""
    search_query = f"{artist} {title}"
    base_url = "https://music.bugs.co.kr"
    # [핵심 수정] 검색 URL을 '가사' 탭으로 변경
    search_url = f"{base_url}/search/lyrics?q={quote(search_query)}"

    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 15)

        # '가사' 검색 결과 테이블이 로드될 때까지 대기
        wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'lyrics')]")))

        # [핵심 수정] trackid 속성이 있는 tr만(곡 정보 라인) 순회
        search_results = driver.find_elements(By.XPATH, "//table[contains(@class, 'lyrics')]//tr[@trackid]")

        if not search_results or "검색 결과가 없습니다." in driver.page_source:
            print("  - 벅스 '가사' 검색 결과가 없습니다.")
            return None

        for result in search_results:
            try:
                found_title = result.find_element(By.CSS_SELECTOR, 'p.title a').text
                found_artist = result.find_element(By.CSS_SELECTOR, 'p.artist a').text

                title_score = fuzz.partial_ratio(normalize_string(title), normalize_string(found_title))
                artist_score = fuzz.partial_ratio(normalize_string(artist), normalize_string(found_artist))

                if title_score >= SIMILARITY_THRESHOLD and artist_score >= SIMILARITY_THRESHOLD:
                    print(f"  ➡️ '가사' 검색에서 일치하는 곡 발견! (유사도: 제목 {title_score}%, 아티스트 {artist_score}%)")

                    # [핵심 수정] 가장 안정적인 '곡정보' 링크를 사용
                    track_info_link_element = result.find_element(By.CSS_SELECTOR, 'a.trackInfo')
                    relative_link = track_info_link_element.get_attribute('href')

                    if not relative_link or "javascript" in relative_link:
                        continue

                    song_link_full = urljoin(base_url, relative_link)
                    print(f"  - 상세 페이지로 이동: {song_link_full}")
                    driver.get(song_link_full)

                    # 상세 페이지가 로드될 때까지 대기 (가사 섹션을 기준으로)
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section.lyrics')))

                    lyrics = ""
                    translation = ""

                    try:
                        # [핵심 수정] <p> 태그 내부에 <xmp>가 있는 구조에 대응
                        lyrics_element = driver.find_element(By.XPATH, '//div[@class="lyricsContainer"]//xmp')
                        lyrics = lyrics_element.get_attribute("innerHTML").strip()

                        try:
                            # 번역 가사가 있는 경우 수집 시도
                            translation = driver.find_element(By.XPATH, '//div[@class="lyricsContainer"]//div[@class="translateContainer"]').text.strip()
                        except NoSuchElementException:
                            pass # 번역은 없어도 괜찮음

                    except NoSuchElementException:
                        # 가사가 <xmp>에 없는 다른 경우들 처리 (예: 비공개)
                        try:
                            not_open_msg_element = driver.find_element(By.CSS_SELECTOR, 'div.lyricsContainer p')
                            if "권리사의 요청" in not_open_msg_element.text:
                                lyrics = "[권리사 요청으로 가사 비공개]"
                                print("  - 권리사 요청으로 가사가 비공개된 곡입니다.")
                            else:
                                lyrics = "[가사 없음]"
                                print("  - 가사 정보가 없어 [가사 없음]으로 처리합니다.")
                        except NoSuchElementException:
                            lyrics = "[가사 없음]"
                            print("  - 가사 요소를 찾을 수 없어 [가사 없음]으로 처리합니다.")

                    first_line = lyrics.split('\n')[0].strip()
                    print(f"  🎶 성공! 가사 첫 줄: {first_line}")

                    return {'source': 'Bugs', 'lyrics': lyrics, 'translation': translation}
            except NoSuchElementException:
                continue
    except Exception as e:
        print(f"  - 벅스 검색/수집 중 오류 발생: {e}")
        return None

    print("  - 벅스에서 최종 일치 곡을 찾지 못했습니다.")
    return None

def scrape_from_utanet(driver, artist, title):
    """[안정화 버전] Uta-net 수집 로직"""
    base_url = "https://www.uta-net.com"
    # 제목으로만 검색하고, 아티스트는 결과 내에서 비교
    search_url = f"{base_url}/search/?Keyword={quote(title)}&Aselect=2&Bselect=3"
    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 10)
        try:
            # 검색 결과 없음 메시지 먼저 확인
            driver.find_element(By.CSS_SELECTOR, "div.error_msg")
            print("  - Uta-Net에서 검색 결과가 없습니다.")
            return None
        except NoSuchElementException:
            # 검색 결과가 있으면 테이블 로딩 대기
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.result_table tbody tr")))
            search_results = driver.find_elements(By.CSS_SELECTOR, "div.result_table tbody tr")

            for result in search_results:
                try:
                    found_title_element = result.find_element(By.CSS_SELECTOR, 'td.side.td1 a')
                    found_title = found_title_element.text
                    found_artist = result.find_element(By.CSS_SELECTOR, 'td.side.td2 a').text

                    title_score = fuzz.partial_ratio(normalize_string(title), normalize_string(found_title))
                    artist_score = fuzz.partial_ratio(normalize_string(artist), normalize_string(found_artist))

                    if title_score >= 60 and artist_score >= 60:
                        print(f"  ➡️ Uta-Net에서 일치하는 곡 발견! (유사도: 제목 {title_score}%, 아티스트 {artist_score}%)")
                        relative_link = found_title_element.get_attribute('href')
                        song_link_full = urljoin(base_url, relative_link)

                        driver.get(song_link_full)
                        wait.until(EC.presence_of_element_located((By.ID, "kashi_area")))
                        lyrics = driver.find_element(By.ID, "kashi_area").text.strip()

                        first_line = lyrics.split('\n')[0].strip() if lyrics else "가사 정보 없음"
                        print(f"  🎶 성공! 가사 첫 줄: {first_line}")

                        return {'source': 'Uta-Net', 'lyrics': lyrics, 'translation': ''}
                except NoSuchElementException:
                    continue
    except Exception as e:
        print(f"  - Uta-Net 검색/수집 중 오류 발생: {e}")
        return None

    print("  - Uta-Net에서 최종 일치 곡을 찾지 못했습니다.")
    return None


# ==============================================================================
# 4. 메인 실행 로직
# ==============================================================================
print("1단계: 데이터 사전 준비 시작...")
if not os.path.exists(INPUT_FILE):
    print(f"❌ 오류: 원본 파일 '{INPUT_FILE}'을 찾을 수 없습니다. 파일을 업로드해주세요.")
else:
    df = pd.read_csv(INPUT_FILE)
    unique_songs_df = df[['artist', 'title']].drop_duplicates().reset_index(drop=True)
    unique_songs_df.to_csv(UNIQUE_SONG_LIST_FILE, index=False, encoding='utf-8-sig')
    print(f"✅ 원본 {len(df)}곡 중, 고유 곡 {len(unique_songs_df)}개를 '{UNIQUE_SONG_LIST_FILE}' 파일로 저장했습니다.")

    print("\n2단계: 수집 대상 목록 준비 시작...")

    # 이전에 성공한 곡 목록
    try:
        completed_df = pd.read_csv(OUTPUT_FILE)
        completed_songs = set(zip(completed_df['artist'], completed_df['title']))
        print(f"📄 기존 수집 파일 '{OUTPUT_FILE}'에서 {len(completed_songs)}개의 성공 데이터를 확인했습니다.")
    except (FileNotFoundError, pd.errors.EmptyDataError):
        completed_df = pd.DataFrame()
        completed_songs = set()
        print(f"✨ 새 수집 파일 '{OUTPUT_FILE}'을 생성하고 전체 수집을 시작합니다.")
        pd.DataFrame(columns=['artist', 'title', 'source', 'lyrics', 'translation']).to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    # 전체 고유 목록에서 성공한 곡들을 제외하여 수집할 목록 생성
    unique_songs_df['key'] = list(zip(unique_songs_df['artist'], unique_songs_df['title']))
    to_scrape_df = unique_songs_df[~unique_songs_df['key'].isin(completed_songs)].drop(columns=['key'])

    if to_scrape_df.empty:
        print("\n🎉 모든 곡의 가사 수집이 이미 완료되었습니다!")
    else:
        print(f"⏭️ {len(completed_songs)}개는 이미 수집되어 건너뜁니다. 남은 곡: {len(to_scrape_df)}개")
        print(f"\n3단계: 총 {len(to_scrape_df)}곡을 대상으로 수집을 시작합니다...")

        # ===============================================================
        # !!! 시작할 인덱스 번호를 지정합니다 (0부터 시작) !!!
        # 예: 949번째 곡부터 시작하려면 948로 설정
        START_INDEX =948
        # ===============================================================

        # 실패 목록 파일 초기화 (새로운 실행 시)
        if os.path.exists(FAILED_FILE):
            os.remove(FAILED_FILE)

        # iterrows()는 원본 DataFrame의 인덱스를 사용하므로, to_scrape_df를 다시 인덱싱
        to_scrape_df = to_scrape_df.reset_index(drop=True)

        for index, row in tqdm(to_scrape_df.iterrows(), total=len(to_scrape_df), desc="J-POP 가사 수집 중"):
            # 인덱스 체크
            if index < START_INDEX:
                continue

            artist, title = row['artist'], row['title']
            print(f"\n🔍 수집 시도 ({index+1}/{len(to_scrape_df)}): {artist} - {title}")
            driver, scraped_data = None, None
            try:
                driver = setup_driver()
                scraped_data = scrape_from_bugs_final_perfected(driver, artist, title)

                if not scraped_data:
                    scraped_data = scrape_from_utanet(driver, artist, title)

                if scraped_data:
                    new_row = pd.DataFrame([{'artist': artist, 'title': title, **scraped_data}])
                    new_row.to_csv(OUTPUT_FILE, mode='a', header=False, index=False, encoding='utf-8-sig')
                else:
                    print(f"  ❌ 최종 수집 실패. 실패 목록에 기록합니다.")
                    with open(FAILED_FILE, "a", encoding="utf-8") as f:
                        f.write(f"{artist}\t{title}\n")
            except Exception as e:
                print(f"  🚨 루프 내에서 예상치 못한 오류 발생: {e}")
                with open(FAILED_FILE, "a", encoding="utf-8") as f:
                    f.write(f"{artist}\t{title}\n")
            finally:
                if driver:
                    driver.quit()
                time.sleep(random.uniform(2, 4))

        print("\n🎉 모든 가사 수집 작업이 종료되었습니다!")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (2,665 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

J-POP 가사 수집 중:   0%|          | 0/1069 [00:00<?, ?it/s]


🔍 수집 시도 (949/1069): 米津玄師 - Azalea
  🚨 루프 내에서 예상치 못한 오류 발생: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x56e63f9c601a <unknown>
#1 0x56e63f465a70 <unknown>
#2 0x56e63f4a0e07 <unknown>
#3 0x56e63f49b5a7 <unknown>
#4 0x56e63f4eb93e <unknown>
#5 0x56e63f4eaf06 <unknown>
#6 0x56e63f4dd1b3 <unknown>
#7 0x56e63f4a959b <unknown>
#8 0x56e63f4aa971 <unknown>
#9 0x56e63f98b1eb <unknown>
#10 0x56e63f98ef39 <unknown>
#11 0x56e63f9722c9 <unknown>
#12 0x56e63f98fae8 <unknown>
#13 0x56e63f956baf <unknown>
#14 0x56e63f9b30a8 <unknown>
#15 0x56e63f9b3286 <unknown>
#16 0x56e63f9c4ff6 <unknown>
#17 0x7a5d8d7bcac3 <unknown>


🔍 수집 시도 (950/1069): Aqours - 永久hours
  🚨 루프 내에서 예상치 못한 오류 발생: Message: session not created: probably

KeyboardInterrupt: 

In [ ]:
# ==========================================================
# [문제 진단을 위한 단독 테스트 코드]
# ==========================================================
import random
from selenium import webdriver
from selenium.common.exceptions import SessionNotCreatedException

def setup_driver_test():
    """
    세션 충돌 방지 옵션이 포함된 드라이버 설정 함수입니다.
    이 함수가 독립적으로 실행되는지 확인하는 것이 목적입니다.
    """
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # 세션 충돌을 방지하는 핵심 옵션들
    chrome_options.add_argument(f'--user-data-dir=/tmp/chrome_user_data_{random.randint(10000, 99999)}')
    chrome_options.add_argument("--disable-gpu")

    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')

    return webdriver.Chrome(options=chrome_options)

print("드라이버 단독 실행 테스트를 시작합니다...")
driver_instance = None
try:
    driver_instance = setup_driver_test()
    print("✅ 성공: 드라이버 객체가 성공적으로 생성되었습니다.")
    print(f"   - 드라이버 정보: {driver_instance}")

except SessionNotCreatedException as e:
    print("❌ 실패: 드라이버 생성 중 SessionNotCreatedException 오류가 발생했습니다.")
    print("   - 이는 Colab 런타임 환경의 일시적인 문제일 가능성이 높습니다.")
    print(f"   - 전체 오류 메시지: {e}")

except Exception as e:
    print("❌ 실패: 드라이버 생성 중 예상치 못한 다른 오류가 발생했습니다.")
    print(f"   - 오류 유형: {type(e).__name__}")
    print(f"   - 전체 오류 메시지: {e}")

finally:
    if driver_instance:
        driver_instance.quit()
        print("✅ 성공: 생성된 드라이버가 정상적으로 종료되었습니다.")

드라이버 단독 실행 테스트를 시작합니다...
❌ 실패: 드라이버 생성 중 SessionNotCreatedException 오류가 발생했습니다.
   - 이는 Colab 런타임 환경의 일시적인 문제일 가능성이 높습니다.
   - 전체 오류 메시지: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x5c049690601a <unknown>
#1 0x5c04963a5a70 <unknown>
#2 0x5c04963e0e07 <unknown>
#3 0x5c04963db5a7 <unknown>
#4 0x5c049642b93e <unknown>
#5 0x5c049642af06 <unknown>
#6 0x5c049641d1b3 <unknown>
#7 0x5c04963e959b <unknown>
#8 0x5c04963ea971 <unknown>
#9 0x5c04968cb1eb <unknown>
#10 0x5c04968cef39 <unknown>
#11 0x5c04968b22c9 <unknown>
#12 0x5c04968cfae8 <unknown>
#13 0x5c0496896baf <unknown>
#14 0x5c04968f30a8 <unknown>
#15 0x5c04968f3286 <unknown>
#16 0x5c0496904ff6 <unknown>
#17 0x7f7a574dbac3 <unknown>

